# Reformer: Text Generation

This notebook was designed to run on TPU.

To use TPUs in Colab, click "Runtime" on the main menu bar and select Change runtime type. Set "TPU" as the hardware accelerator.

In [15]:
# Install JAX.
!pip install --upgrade jax
!pip install --upgrade jaxlib
!pip install --upgrade trax

# Make sure the Colab Runtime is set to Accelerator: TPU.
import requests
import os
if 'TPU_DRIVER_MODE' not in globals():
  url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/tpu_driver0.1-dev20191206'
  resp = requests.post(url)
  TPU_DRIVER_MODE = 1

# The following is required to use TPU Driver as JAX's backend.
from jax.config import config
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']
print(config.FLAGS.jax_backend_target)

Requirement already up-to-date: jax in /usr/local/lib/python3.6/dist-packages (0.2.7)
Requirement already up-to-date: jaxlib in /usr/local/lib/python3.6/dist-packages (0.1.57+cuda101)
Requirement already up-to-date: trax in /usr/local/lib/python3.6/dist-packages (1.3.6)
grpc://10.86.215.66:8470


In [16]:
!pip install --upgrade -q sentencepiece
!pip install --upgrade -q gin 

from tensorflow.compat.v1.io.gfile import GFile
import gin
import os
import jax
import trax
from trax.data import inputs

import numpy as np
import jax.numpy as jnp

from scipy.special import softmax

from sentencepiece import SentencePieceProcessor

## Setting up data and model

In this notebook, we'll be pushing the limits of just how many tokens we can fit on a single TPU device. The TPUs available in Colab have 8GB of memory per core, and 8 cores. We will set up a Reformer model that can fit a copy of "Crime and Punishment" on *each* of the 8 TPU cores (over 500,000 tokens per 8GB of memory).

In [17]:
# Import a copy of "Crime and Punishment", by Fyodor Dostoevsky
with GFile('gs://trax-ml/reformer/crime-and-punishment-2554.txt') as f:
  text = f.read()

# The file read above includes metadata and licensing information.
# For training our language model, we will only use the actual novel text.
start = text.find('CRIME AND PUNISHMENT')  # skip header
start = text.find('CRIME AND PUNISHMENT', start + 1)  # skip header
start = text.find('CRIME AND PUNISHMENT', start + 1)  # skip translator preface
end = text.rfind('End of Project')  # skip extra text at the end
text = text[start:end].strip()

In [18]:
# Load a BPE vocabulaary with 320 types. This mostly consists of single letters
# and pairs of letters, but it has some common words and word pieces, too.
!gsutil cp gs://trax-ml/reformer/cp.320.* .

TOKENIZER = SentencePieceProcessor()
TOKENIZER.load('cp.320.model')

Copying gs://trax-ml/reformer/cp.320.model...
Copying gs://trax-ml/reformer/cp.320.vocab...
/ [2 files][239.0 KiB/239.0 KiB]                                                
Operation completed over 2 objects/239.0 KiB.                                    


True

In [19]:
# Tokenize
IDS = TOKENIZER.EncodeAsIds(text)
IDS = np.asarray(IDS, dtype=np.int32)
PAD_AMOUNT = 512 * 1024 - len(IDS)
print("Number of tokens:", IDS.shape[0])

Number of tokens: 513812


As we see above, "Crime and Punishment" has just over half a million tokens with the BPE vocabulary we have selected.

Normally we would have a dataset with many examples, but for this demonstration we fit a language model on the single novel only. We don't want the model to just memorize the dataset by encoding the words in its position embeddings, so at each training iteration we will randomly select how much padding to put before the text vs. after it.

We have 8 TPU cores, so we will separately randomize the amount of padding for each core.

In [20]:
# Set up the data pipeline.
def my_inputs(n_devices):
  while True:
    inputs = []
    mask = []
    pad_amounts = np.random.choice(PAD_AMOUNT, n_devices)
    for i in range(n_devices):
      inputs.append(np.pad(IDS, (pad_amounts[i], PAD_AMOUNT - pad_amounts[i]),
                            mode='constant'))
      mask.append(np.pad(np.ones_like(IDS, dtype=np.float32),
                          (pad_amounts[i], PAD_AMOUNT - pad_amounts[i]),
                          mode='constant'))
    inputs = np.stack(inputs)
    mask = np.stack(mask)
    yield (inputs, inputs, mask)

print("(device count, tokens per device) = ",
      next(my_inputs(trax.fastmath.device_count()))[0].shape)

(device count, tokens per device) =  (8, 524288)


In [21]:
# Configure hyperparameters.
gin.parse_config("""
import trax.layers
import trax.models
import trax.optimizers
import trax.data.inputs
import trax.supervised.trainer_lib

# Parameters that will vary between experiments:
# ==============================================================================
train.model = @trax.models.ReformerLM
# Our model will have 6 layers, alternating between the LSH attention proposed
# in the Reformer paper and local attention within a certain context window.
n_layers = 6
attn_type = [
  @trax.layers.SelfAttention,
  @LSHSelfAttention,  
  @trax.layers.SelfAttention,
  @LSHSelfAttention,
  @trax.layers.SelfAttention,
  @LSHSelfAttention,
  ]
share_qk = False  # LSH attention ignores this flag and always shares q & k
n_heads = 2
attn_kv = 64
dropout = 0.05
n_tokens = 524288

# Parameters for multifactor:
# ==============================================================================
multifactor.constant = 0.01
multifactor.factors = 'constant * linear_warmup * cosine_decay'
multifactor.warmup_steps = 100
multifactor.steps_per_cycle = 900

# Parameters for Adam:
# ==============================================================================
Adam.weight_decay_rate=0.0
Adam.b1 = 0.86
Adam.b2 = 0.92
Adam.eps = 1e-9

# Parameters for SelfAttention:
# ==============================================================================
trax.layers.SelfAttention.attention_dropout = 0.05
trax.layers.SelfAttention.chunk_len = 64
trax.layers.SelfAttention.n_chunks_before = 1
trax.layers.SelfAttention.n_parallel_heads = 1

# Parameters for LSHSelfAttention:
# ==============================================================================
LSHSelfAttention.attention_dropout = 0.0
LSHSelfAttention.chunk_len = 64
LSHSelfAttention.n_buckets = [64, 128]
LSHSelfAttention.n_chunks_after = 0
LSHSelfAttention.n_chunks_before = 1
LSHSelfAttention.n_hashes = 1
LSHSelfAttention.n_parallel_heads = 1
LSHSelfAttention.predict_drop_len = 128
LSHSelfAttention.predict_mem_len = 1024

# Parameters for ReformerLM:
# ==============================================================================
ReformerLM.attention_type = %attn_type
ReformerLM.d_attention_key = %attn_kv
ReformerLM.d_attention_value = %attn_kv
ReformerLM.d_model = 256
ReformerLM.d_ff = 512
ReformerLM.dropout = %dropout
ReformerLM.ff_activation = @trax.layers.Relu
ReformerLM.max_len = %n_tokens
ReformerLM.mode = 'train'
ReformerLM.n_heads = %n_heads
ReformerLM.n_layers = %n_layers
ReformerLM.vocab_size = 320
ReformerLM.axial_pos_shape = (512, 1024)
ReformerLM.d_axial_pos_embs= (64, 192)
""")

([],
 ['trax.layers',
  'trax.models',
  'trax.optimizers',
  'trax.data.inputs',
  'trax.supervised.trainer_lib'])

In [22]:
# Set up a Trainer.
output_dir = os.path.expanduser('~/train_dir/')
!rm -f ~/train_dir/model.pkl.gz  # Remove old model

trainer = trax.supervised.Trainer(
    model=trax.models.ReformerLM,
    loss_fn=trax.layers.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam,
    lr_schedule=trax.lr.multifactor(),
    inputs=trax.data.inputs.Inputs(my_inputs),
    output_dir=output_dir)

In [23]:
# Run one training step, to make sure the model fits in memory.
# The first time trainer.train_epoch is called, it will JIT the entire network
# architecture, which takes around 2 minutes. The JIT-compiled model is saved
# so subsequent runs will be much faster than the first.
trainer.train_epoch(n_steps=1, n_eval_steps=1)


Step      1: Ran 1 train steps in 158.46 secs
Step      1: Evaluation
Step      1: train                   accuracy |  0.00292933
Step      1: train                       loss |  6.40892601
Step      1: train         neg_log_perplexity | -6.40892601
Step      1: train          sequence_accuracy |  0.00000000
Step      1: train weights_per_batch_per_core |  513812.00000000
Step      1: eval                    accuracy |  0.00286681
Step      1: eval                        loss |  6.40883303
Step      1: eval          neg_log_perplexity | -6.40883303
Step      1: eval           sequence_accuracy |  0.00000000
Step      1: eval  weights_per_batch_per_core |  513812.00000000
Step      1: Finished evaluation


In [24]:
# Train for 600 steps total
# The first ~20 steps are slow to run, but after that it reaches steady-state
# speed. This will take at least 30 minutes to run to completion, but can safely
# be interrupted by selecting "Runtime > Interrupt Execution" from the menu.
# The language model won't be exceptionally good when trained for just a few
# steps and with minimal regularization. However, we can still sample from it to
# see what it learns.
trainer.train_epoch(n_steps=9, n_eval_steps=1)
for _ in range(59):
  trainer.train_epoch(n_steps=10, n_eval_steps=1)


Step     10: Ran 9 train steps in 187.68 secs
Step     10: Evaluation
Step     10: train                   accuracy |  0.02779835
Step     10: train                       loss |  5.37467575
Step     10: train         neg_log_perplexity | -5.37467575
Step     10: train          sequence_accuracy |  0.00000000
Step     10: train weights_per_batch_per_core |  513812.00000000
Step     10: eval                    accuracy |  0.02782243
Step     10: eval                        loss |  5.37470198
Step     10: eval          neg_log_perplexity | -5.37470198
Step     10: eval           sequence_accuracy |  0.00000000
Step     10: eval  weights_per_batch_per_core |  513812.00000000
Step     10: Finished evaluation

Step     20: Ran 10 train steps in 35.91 secs
Step     20: Evaluation
Step     20: train                   accuracy |  0.02764411
Step     20: train                       loss |  5.30323601
Step     20: train         neg_log_perplexity | -5.30323601
Step     20: train          sequenc

## Sample from the model

In [25]:
# As we report in the Reformer paper, increasing the number of hashing rounds
# helps with quality. We can even increase the number of hashing rounds at
# evaluation time only.

gin.parse_config("""LSHSelfAttention.n_hashes = 4""")

([], [])

In [26]:
# Load the trained Reformer in 'predict' mode
model = trax.models.ReformerLM(mode='predict')
model.init_from_file(os.path.join(output_dir,'model.pkl.gz'),
                     weights_only=True)

# Sample from ReformerLM
output_token_ids = trax.supervised.decoding.autoregressive_sample(
    model, temperature=0.0)

# Decode token IDs
# Reformer outputed a batch with one item, we access it using [0]
# tolist() converts from int64 to int, the type SentencePiece expects
TOKENIZER.DecodeIds(output_token_ids[0].tolist()) 


'CHAPTER I OR’t’s beon, _wo, it’s not go!” seemed stepped short instant. Nastasya, but he did not know it was away. He was a garleep, and he had a flat closed by the hands, and he had not evening, and he had'